In [39]:
import zipfile
import os
import xlwings as xw
import psutil
import os
import shutil
import zipfile
from pathlib import Path


not_table_file_types = ['pdf', 'jpeg', 'jpg', 'png', 'zip', 'PDF', 'JPEG', 'JPG', 'PNG', 'ZIP']
arch_file_types = ['zip', 'ZIP']
table_file_types = ['xlsx', 'xlsm', 'xls', 'ods', 'XLSX', 'XLSM', 'XLS', 'ODS']


main_dir = str(Path(os.getcwd()).parent)+'\\'
current_dir = os.getcwd()+'\\'

main_dir, current_dir

extract_dir = main_dir + 'unzip\\'
sort_dir = main_dir + 'sorted\\'
not_table_kjz_dir = sort_dir + 'КЖЗ\\'
tables_dir = sort_dir + 'Контроль МО\\'
others_files_dir = sort_dir + 'Контроль МО - не таблицы\\'

try:
    os.makedirs(sort_dir)
    os.makedirs(extract_dir)
except: pass

In [40]:
# создать папки для сортировки
try:
    os.makedirs(not_table_kjz_dir)
    os.makedirs(tables_dir)
    os.makedirs(others_files_dir)
except: pass

In [41]:
# перед формированием словаря, где ключ = путь_к_паке, а значения = список имен файлов в папке
# формирование словаря, где ключ = путь_к_паке, а значения = список имен файлов в папке
def unpacked_tree(extract_dir=extract_dir):
    unpacked_dict = {}
    unpacked_list = []
    unpacked_dir_tree = os.walk(extract_dir)
    for i in unpacked_dir_tree:
        dir_structure = list(i)
        if dir_structure[-1]:
            unpacked_dict[dir_structure[0]+'\\'] = dir_structure[-1]
            for f in dir_structure[-1]:
                unpacked_list.append(dir_structure[0]+ '\\' + f)
    return unpacked_dict, unpacked_list

In [42]:
########### DEF unpack_zipfile #################################
def unpack_zipfile(dir_with_archive=main_dir, extract_dir=extract_dir, encoding='cp866'):
    for file in os.listdir(dir_with_archive):
        if '.zip' in file:
            with zipfile.ZipFile(dir_with_archive+file) as archive:
                for entry in archive.infolist():
                    try:
                        name = entry.filename.encode('cp437').decode(encoding)
                    except:
                        name = entry.filename.encode('cp866').decode('cp866')
                    target = os.path.join(extract_dir, *name.split('/'))    
                    os.makedirs(os.path.dirname(target), exist_ok=True)
                    if not entry.is_dir():  # file
                        with archive.open(entry) as source, open(target, 'wb') as dest:
                            shutil.copyfileobj(source, dest)
            os.remove(dir_with_archive+file)

In [43]:
unpack_zipfile(main_dir)

In [44]:
unpacked_dict, unpacked_list = unpacked_tree(extract_dir)

In [47]:
# поиск архивов в папке unzip
# и разархивирование их в папку 'unzip' 
def find_nested_zip(unpacked_dict=unpacked_dict, extract_dir=extract_dir):
    try:
        for k,v in unpacked_dict.items():
            res = [string for string in v if '.zip' in string]
            for _ in res:
                match = os.path.join(k, _)
                match = os.path.abspath(match)
                if match:
                    unpack_zipfile(k, extract_dir=extract_dir, encoding='cp866')
                    os.remove(os.path.abspath(match))
    except:
        print('no zip in dir')

In [48]:
find_nested_zip(unpacked_dict, extract_dir)

no zip in dir


In [49]:
unpacked_dict, unpacked_list = unpacked_tree(extract_dir)

In [50]:
def rename_files_by_folder(unpacked_list=unpacked_list, extract_dir=extract_dir):
    for path_to_file in unpacked_list:
        name_listing = path_to_file.split('\\')
        res = any('кжз' in string.lower() for string in name_listing)
        kjz_str = 'КЖЗ-'
        if res:
            name_listing[-1] = kjz_str+name_listing[-2]+'-'+name_listing[-1]
        else:
            name_listing[-1] = name_listing[-2]+'-'+name_listing[-1]

        new_path_to_file = '\\'.join([str(n) for n in name_listing]).replace(kjz_str*2,
                                                                             kjz_str)
        new_path_to_file = new_path_to_file.replace(name_listing[-2]+'-'+name_listing[-2],
                                                    name_listing[-2])
        os.rename(path_to_file,
                  new_path_to_file)

In [51]:
rename_files_by_folder(unpacked_list)

In [52]:
unpacked_dict, unpacked_list = unpacked_tree(extract_dir)

In [53]:
def sort_files(extract_dir=extract_dir,
               unpacked_list=unpacked_list,
               not_table_file_types=not_table_file_types,
               table_file_types=table_file_types,
               not_table_kjz_dir=not_table_kjz_dir,
               tables_dir=tables_dir,
               others_files_dir=others_files_dir):
    
    for path_to_file in unpacked_list:
        name_listing = path_to_file.split('\\')
        res = any('кжз' in string.lower() for string in name_listing)
        try:
            if res:
                if path_to_file.endswith(tuple(not_table_file_types)):
                    os.rename(path_to_file, not_table_kjz_dir+name_listing[-1])
                else: os.rename(path_to_file, tables_dir+name_listing[-1])
            elif path_to_file.endswith(tuple(table_file_types)):
                os.rename(path_to_file, tables_dir+name_listing[-1])
            else:
                path_to_file.endswith(tuple(not_table_file_types))
                os.rename(path_to_file, others_files_dir+name_listing[-1])
        except:
            print(path_to_file)
    shutil.rmtree(extract_dir)

In [54]:
sort_files()

In [337]:
for path_to_file in unpacked_list:
    name_listing = path_to_file.split('\\')
    res = any('кжз' in string.lower() for string in name_listing)
#     try:
    if res:
        print(path_to_file)
#             if path_to_file.endswith(tuple(not_table_file_types)):
#                 os.rename(path_to_file, not_table_kjz_dir+name_listing[-1])
#             else: os.rename(path_to_file, tables_dir+name_listing[-1])
#         elif path_to_file.endswith(tuple(table_file_types)):
#             os.rename(path_to_file, tables_dir+name_listing[-1])
#         else:
#             path_to_file.endswith(tuple(not_table_file_types))
#             os.rename(path_to_file, others_files_dir+name_listing[-1])
#     except:
#         print(path_to_file)

C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\unzip\02.06.2022 КЖЗ\КЖЗ-02.06.2022 КЖЗ-01.06.22_ГП_Контроль_МО.xlsx
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\unzip\02.06.2022 КЖЗ\КЖЗ-02.06.2022 КЖЗ-смирнова.pdf
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\unzip\Контроль МО (1)\Контроль МО\01.06.2022 КЖЗ\КЖЗ-01.06.2022 КЖЗ-31.05.2022_Контроль_МО.xlsx
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\unzip\Контроль МО (1)\Контроль МО\01.06.2022 КЖЗ\КЖЗ-01.06.2022 КЖЗ-глушко.pdf
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\unzip\Контроль МО (1)\Контроль МО\01.06.2022 КЖЗ\КЖЗ-01.06.2022 КЖЗ-зеленова.pdf
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\unzip\Контроль МО (1)\Контроль МО\01.06.2022 КЖЗ\КЖЗ-01.06.2022 КЖЗ-зеленова1.pdf
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\unzip\Контроль МО (1)\Контроль МО\01.06.2022 КЖЗ\КЖЗ-01.06.2022 КЖЗ-ким.pdf
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Next

In [ ]:
def sort_folder(unpacked_list, sort_dir):
    for path_to_file in unpacked_list:
        res = any('кжз' in string.lower() for string in name_listing)
        if table_file_types not in res:
            print(res)
#             current_path = os.path.abspath(path_to_file)
#             new_path = 
#             os.replace(os.path.join(path, name), os.path.join(path, method, name))
#     if file.split('.')[-1] in ['pdf', 'jpeg', 'jpg', 'png', 'zip', 'PDF', 'JPEG', 'JPG', 'PNG', 'ZIP']:
#         replacer(path, file, 'Контроль_МО-Прочие_отсортированные_файлы')

In [ ]:
#     kjz_paths = []
#     kjz_path = ["\\".join(name_listing) for s in name_listing if 'КЖЗ' in s]
#     if kjz_path:
#         kjz_paths.append()
# #         print(path_to_file)
#         name_listing[-1] = 'КЖЗ-'+name_listing[-2]+'-'+name_listing[-1]
#         name_listing[-1] = name_listing[-1].replace('КЖЗ-КЖЗ-', 'КЖЗ-')
#         os.rename(path_to_file,
#           '\\'.join([str(n) for n in name_listing]))
#     else:
#         name_listing[-1] = name_listing[-2]+'-'+name_listing[-1]
#         os.rename(path_to_file,
#                   '\\'.join([str(n) for n in name_listing]))

In [ ]:
path_to_file = 'C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\unzip\\Контроль МО\\20.05.2022\\КЖЗ 19.05.2022\\19.05.2022\\некрасова.pdf'
name_listing = path_to_file.split('\\')
res = any('кжз' in string.lower() for string in name_listing)
print(name_listing)
if res:
    name_listing[-1] = 'КЖЗ-'+name_listing[-2]+'-'+name_listing[-1]
    print(name_listing)
else: print('no')

In [54]:
for path_to_file in unpacked_list:
    if 'кжз' in path_to_file.lower():
        print(path_to_file)
        abs_path_to_file = os.path.abspath(path_to_file)
        new_abs_path_to_file = abs_path_to_file.replace(r'\.*$', )
#         os.rename(abs_path_to_file, '\\'.join([str(n) for n in name_listing]))

C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\Nextcloud_connector\unzip\КЖЗ\test1-ЦАОП-ГКОБ№1-ЦАО-цитологическое-исследование-06.05.2022.pdf
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\Nextcloud_connector\unzip\КЖЗ\test2-ЦАОП-ГКОБ№1-ЦАО-цитологическое-исследование-06.05.2022.pdf
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\Nextcloud_connector\unzip\КЖЗ\test3-ЦАОП-ГКОБ№1-ЦАО-цитологическое-исследование-06.05.2022.pdf
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\Nextcloud_connector\unzip\Контроль МО\20.05.2022\КЖЗ 19.05.2022\19.05.2022\19.05.2022_ГП_контроль_МО_____КЖЗ.xlsx
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\Nextcloud_connector\unzip\Контроль МО\20.05.2022\КЖЗ 19.05.2022\19.05.2022\игнатова.pdf
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\Nextcloud_connector\unzip\Контроль МО\20.05.2022\КЖЗ 19.05.2022\19.05.2022\москаленко.pdf
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\Nex

In [52]:
for path_to_file in unpacked_list:
    name_listing = path_to_file.split('\\')
    print(name_listing)
#     kjz_paths = []
#     kjz_path = ["\\".join(name_listing) for s in name_listing if 'КЖЗ' in s]
#     if kjz_path:
#         kjz_paths.append()
# #         print(path_to_file)
#         name_listing[-1] = 'КЖЗ-'+name_listing[-2]+'-'+name_listing[-1]
#         name_listing[-1] = name_listing[-1].replace('КЖЗ-КЖЗ-', 'КЖЗ-')
#         os.rename(path_to_file,
#           '\\'.join([str(n) for n in name_listing]))
#     else:
#         name_listing[-1] = name_listing[-2]+'-'+name_listing[-1]
#         os.rename(path_to_file,
#                   '\\'.join([str(n) for n in name_listing]))

['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO', 'Nextcloud_connector', 'Nextcloud_connector', 'unzip', 'КЖЗ', 'test1-ЦАОП-ГКОБ№1-ЦАО-цитологическое-исследование-06.05.2022.pdf']
['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO', 'Nextcloud_connector', 'Nextcloud_connector', 'unzip', 'КЖЗ', 'test2-ЦАОП-ГКОБ№1-ЦАО-цитологическое-исследование-06.05.2022.pdf']
['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO', 'Nextcloud_connector', 'Nextcloud_connector', 'unzip', 'КЖЗ', 'test3-ЦАОП-ГКОБ№1-ЦАО-цитологическое-исследование-06.05.2022.pdf']
['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO', 'Nextcloud_connector', 'Nextcloud_connector', 'unzip', 'Контроль МО', '20.05.2022', '19.05.2022_ГП_52 контроль_МО.xlsx']
['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO', 'Nextcloud_connector', 'Nextcloud_connector', 'unzip', 'Контроль МО', '20.05.2022', '19.05.2022_ЦАОП им. А.С. Логинова_контроль_МО.xlsx']
['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO

In [19]:
unpacked_list

['C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\unzip\\КЖЗ\\test1-ЦАОП-ГКОБ№1-ЦАО-цитологическое-исследование-06.05.2022.pdf',
 'C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\unzip\\КЖЗ\\test2-ЦАОП-ГКОБ№1-ЦАО-цитологическое-исследование-06.05.2022.pdf',
 'C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\unzip\\КЖЗ\\test3-ЦАОП-ГКОБ№1-ЦАО-цитологическое-исследование-06.05.2022.pdf',
 'C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\unzip\\Контроль МО\\20.05.2022\\19.05.2022_ГП_52 контроль_МО.xlsx',
 'C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\unzip\\Контроль МО\\20.05.2022\\19.05.2022_ЦАОП им. А.С. Логинова_контроль_МО.xlsx',
 'C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\unzip\\Контроль МО\\20.05.2022\\20.05.2022 ГП 214 пациенты с подозрени

In [11]:
for k,v in unpacked_dict.items(): 
#     name_listing = k.split('\\')
    res = k.split('\\')
    if 'КЖЗ' in res:
        
    print(res)
#     res = [string for string in k if 'КЖЗ' in string]
#     for _ in res:
        
#         match = os.path.join(k, _)
#         match = os.path.abspath(match)
#         if match:
#             unpack_zipfile(k, extract_dir='unzip', encoding='cp866')
#             os.remove(os.path.abspath(match))

['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO', 'Nextcloud_connector', 'Nextcloud_connector', 'unzip', 'КЖЗ', '']
['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO', 'Nextcloud_connector', 'Nextcloud_connector', 'unzip', 'Контроль МО', '20.05.2022', '']
['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO', 'Nextcloud_connector', 'Nextcloud_connector', 'unzip', 'Контроль МО', '20.05.2022', 'КЖЗ 19.05.2022', '19.05.2022', '']
['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO', 'Nextcloud_connector', 'Nextcloud_connector', 'unzip', 'Контроль МО', '23.05.2022', '']
['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO', 'Nextcloud_connector', 'Nextcloud_connector', 'unzip', 'Контроль МО', '23.05.2022', 'КЖЗ', '20.05.2022', '']


In [63]:
path_to_file = 'C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\unzip\\Контроль МО\\20.05.2022\\КЖЗ 19.05.2022\\19.05.2022\\некрасова.pdf'
name_listing = path_to_file.split('\\')
res = any('кжз' in string.lower() for string in name_listing)
print(name_listing)
if res:
    name_listing[-1] = 'КЖЗ-'+name_listing[-2]+'-'+name_listing[-1]
    print(name_listing)
else: print('no')
#     res = any('КЖЗ' in string for string in name_listing)
#     name_listing[-1] = 'КЖЗ-'+name_listing[-2]+'-'+name_listing[-1]
#     print('\\'.join([str(n) for n in name_listing]))
# else:
#     name_listing[-1] = name_listing[-2]+'-'+name_listing[-1]
#     print('\\'.join([str(n) for n in name_listing]))
    
# os.rename(path_to_file,
#           '\\'.join([str(n) for n in name_listing]))

['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO', 'Nextcloud_connector', 'Nextcloud_connector', 'unzip', 'Контроль МО', '20.05.2022', 'КЖЗ 19.05.2022', '19.05.2022', 'некрасова.pdf']
['C:', 'Users', 'KorneevSN', 'Downloads', 'КНИГИ', 'REPO', 'Nextcloud_connector', 'Nextcloud_connector', 'unzip', 'Контроль МО', '20.05.2022', 'КЖЗ 19.05.2022', '19.05.2022', 'КЖЗ-19.05.2022-некрасова.pdf']


In [113]:

            

for path_to_file in list_of_target_files_path:
    name_listing = path_to_file.split('\\')
    if 'КЖЗ' in name_listing:
        res = any('кжз' in string.lower() for string in name_listing)
        name_listing[-1] = 'КЖЗ-'+name_listing[-2]+'-'+name_listing[-1]
    else:
        name_listing[-1] = name_listing[-2]+'-'+name_listing[-1]
    os.rename(path_to_file,
              '\\'.join([str(n) for n in name_listing]))

In [76]:
# for path_to_file in list_of_target_files_path:
#     name_listing = path_to_file.split('\\')
#     name_listing[-1] = 'кжз-'+name_listing[-1]
#     os.rename(path_to_file,
#               '\\'.join([str(n) for n in name_listing]))

to do

1. распаковать архивы найденные в папках
2. маркировать файлы КЖЗ
2. сортировка файлов по папкам по расширению и содержимому таблиц


In [186]:
source = current_dir+'unzip'+'\\'
for _ in os.listdir(source):
    shutil.move(source+'\\'+_, 'unpack') 

Error: Destination path 'C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\Nextcloud_connector\unpacked\Контроль МО' already exists

In [180]:
########### done modules #################################
import os
import shutil
import zipfile

########### DEF unpack_zipfile #################################
def unpack_zipfile(filename, extract_dir, encoding='cp866'):
    with zipfile.ZipFile(filename) as archive:
        for entry in archive.infolist():
            try:
                name = entry.filename.encode('cp437').decode(encoding)
            except:
                name = entry.filename.encode('cp866').decode('cp866')
            if '.zip' in name:
                print('entry=',entry, 'name=', name, sep='\n')
                nested_archive = archive.getinfo(entry.filename)
                path_nested_archive = os.path.abspath(nested_archive.filename)              
                zipfile.ZipFile(path_nested_archive).extract(path_nested_archive, 'unpacked')
                print('nested_archive=',nested_archive, 'path_nested_archive=', path_nested_archive, sep='\n')
#                 with zipfile.ZipFile(filename) as archive:
#             else:          
#                 pass



#   docx_zip = zipfile.ZipFile(zip_path)
# docx_info = docx_zip.getinfo(name_of_image_in_zip)
# path_to_image = os.path.abspath(docx_info.filename)          
            
            

#                 # don't extract absolute paths or ones with .. in them
#                 if name.startswith('/'):
#                     print('dont extract',name)

#                 target = os.path.join(extract_dir, *name.split('/'))    
#                 os.makedirs(os.path.dirname(target), exist_ok=True)
#                 if not entry.is_dir():  # file
#                     with archive.open(entry) as source, open(target, 'wb') as dest:
#                         shutil.copyfileobj(source, dest)

########### RUN unpack_zipfile #################################
unpack_zipfile(main_dir+'23.05.2022-15.01-Контроль МО.zip', 'unpacked', encoding='cp866')

entry=
<ZipInfo filename='è«¡Γα«½∞ îÄ/èåç.zip' external_attr=0x20 file_size=738950>
name=
Контроль МО/КЖЗ.zip


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\è«¡Γα«½∞ îÄ\\èåç.zip'

In [149]:
def old_unzip(file, unpack_dir, exceptions_unpack_dir):
    with zipfile.ZipFile(file) as zip:
    
        for zip_info in zip.infolist():
            if zip_info.filename[-1] == '/':
                continue
                try:
    #                 print(abs_path_str.encode('cp437').decode('cp866'))
                    name = zip_info.filename.encode('cp437').decode('cp866')
                except: 
    #                 print('ex== ',abs_path_str.encode('cp866').decode('cp866'))
                    name = zip_info.filename.encode('cp866').decode('cp866')
    
            if '.zip' in abs_path_str:
                with zipfile.ZipFile(abs_path_str) as nested_zip:
                    for nested_zip_info in nested_zip.infolist():
                        try:
            #                 print(abs_path_str.encode('cp437').decode('cp866'))
                            nested_abs_path_str = nested_zip_info.filename.encode('cp437').decode('cp866')
                        except: 
            #                 print('ex== ',abs_path_str.encode('cp866').decode('cp866'))
                            nested_abs_path_str = nested_zip_info.filename.encode('cp866').decode('cp866')
                    if '/КЖЗ' in abs_path_str and abs_path_str not in table_file_types:
                        abs_path_str = os.path.basename(abs_path_str)
                        zip.extract(zip_info, exceptions_unpack_dir)
                    elif abs_path_str[-1] == '/':
                        continue
                    name = os.path.basename(abs_path_str)
                    zip.extract(zip_info, unpack_dir)
                
            else:
                if '/КЖЗ' in abs_path_str and abs_path_str not in table_file_types:
                    abs_path_str = os.path.basename(abs_path_str)
                    zip.extract(zip_info, exceptions_unpack_dir)
                elif abs_path_str[-1] == '/':
                    continue
                
            name = os.path.basename(abs_path_str)
            zip.extract(zip_info, unpack_dir)

In [150]:
for _ in os.listdir(main_dir):
    if '.zip' in _:
        old_unzip(main_dir+_, 'old_unzip_script', 'exceptions_unpack_dir')

FileNotFoundError: [Errno 2] No such file or directory: 'Контроль МО/КЖЗ.zip'

In [64]:
########### subDEF unpack_zipfile #################################
def unpack_zipfile(filename,
                   extract_dir='unzip',
                   exceptions_dir='unzip_exceptions'
                   encoding='cp866'):
    with zipfile.ZipFile(filename) as archive:
        for entry in archive.infolist():
            try:
                name = entry.filename.encode('cp437').decode(encoding)  # reencode!!!
            except:
                name = entry.filename.encode('cp866').decode(encoding)  # reencode!!!
            print(name)

            # don't extract absolute paths or ones with .. in them
            if 'КЖЗ' in name:
                pass
#             if name.startswith('/') or '..' in name:
#                 continue

            target = os.path.join(extract_dir, *name.split('/'))    
            os.makedirs(os.path.dirname(target), exist_ok=True)
            if not entry.is_dir():  # file
                with archive.open(entry) as source, open(target, 'wb') as dest:
                    shutil.copyfileobj(source, dest)

In [280]:
# for _ in os.listdir(main_dir):
#     if '.zip' in _:
#         unpack_zipfile(main_dir + _)

In [61]:
########### DEF find_dir_by_name_and_extract_files #################################
def find_dir_by_name_and_extract_files(processing_dir_path,
                                       target_dir_name = '\КЖЗ\\',
                                       dir_for_replace = '',
                                       replacing_dir_name = '\КЖЗ\\'):
    """
    0) *dir_for_replace* - print path to parent folder for *replacing_dir_name* !!!
       default 'None' - *replacing_dir_name* will be replaced to *processing_dir_path*   
       
    1) Find subdir in *processing_dir_path* by *target_dir_name*,
    2) if need create *replacing_dir_name* as subdir in *processing_dir_path*
    2) replace files in *target_dir_name* to *replacing_dir_name* 
    """
    
#     os.chdir(processing_dir_path)
    for root, dirs, files in os.walk(os.getcwd(), topdown = False):
        for fld in dirs:
            abs_path_to_folder = os.path.join(root, fld)
            
            print(abs_path_to_folder)

            if target_dir_name in abs_path_to_folder:# and not_table_file_types in abs_path_to_folder:
                if dir_for_replace != '':
#                     print(dir_for_replace)
                    shutil.move(abs_path_to_folder, dir_for_replace)
                else: shutil.move(abs_path_to_folder, processing_dir_path +'\\'+ replacing_dir_name)

In [281]:
# for _ in os.listdir(current_dir + 'unzip\\'):
#     find_dir_by_name_and_extract_files(processing_dir_path = current_dir + 'unzip\\',
#                                        target_dir_name = 'КЖЗ',
#                                        dir_for_replace = current_dir + 'replace_kjz\\',
#                                        replacing_dir_name = 'КЖЗ')

In [71]:
def unzip(file, unpack_dir):
    with zipfile.ZipFile(file) as zip:
        for zip_info in zip.infolist():
            if 'КЖЗ' in zip_info.filename:
                pass
            if zip_info.filename[-1] == '/':
                continue
            zip_info.filename = os.path.basename(zip_info.filename)
            zip.extract(zip_info, unpack_dir)

In [282]:
# with zipfile.ZipFile(main_dir+'23.05.2022-15.01-Контроль МО.zip') as zip:
#     for zip_info in zip.infolist():
#         print(zip_info)

In [72]:
unzip(main_dir+'23.05.2022-15.01-Контроль МО.zip', 'old_arc')

In [64]:
########### DEF find zip file in parent dir and unpack_zipfile #################################
def get_archive_from_main_dir(main_dir):
    for _ in os.listdir(main_dir):
        if '.zip' in _:
            unpack_zipfile(main_dir+_)

In [65]:
get_archive_from_main_dir(main_dir)

In [66]:
unzip_folder = current_dir+'unzip\\'
unzip_folder

'C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\unzip\\'

In [67]:
########### DEF find_dir_by_name_and_extract_files #################################
def find_dir_by_name_and_extract_files(processing_dir_path,
                                       target_dir_name = 'КЖЗ',
                                       dir_for_replace = '',
                                       replacing_dir_name = 'КЖЗ'):
    """
    0) *dir_for_replace* - print path to parent folder for *replacing_dir_name* !!!
       default 'None' - *replacing_dir_name* will be replaced to *processing_dir_path*   
       
    1) Find subdir in *processing_dir_path* by *target_dir_name*,
    2) if need create *replacing_dir_name* as subdir in *processing_dir_path*
    2) replace files in *target_dir_name* to *replacing_dir_name* 
    """
    
#     os.chdir(processing_dir_path)
    for root, dirs, files in os.walk(os.getcwd(), topdown = False):
        for fld in dirs:
            abs_path_to_folder = os.path.join(root, fld)
            
            print(abs_path_to_folder)

            if target_dir_name in abs_path_to_folder.split('\\')[-1] == target_dir_name:
                if dir_for_replace != '':
#                     print(dir_for_replace)
                    shutil.move(abs_path_to_folder, dir_for_replace)
                else: shutil.move(abs_path_to_folder, processing_dir_path +'\\'+ replacing_dir_name)

In [283]:
# ########### RUN find_dir_by_name_and_extract_files #################################
# find_dir_by_name_and_extract_files(unzip_folder)